In [7]:
!pip install wget --quiet
!pip install sentence-transformers --quiet

In [8]:
import wget
import os 

csv_file_path = "https://raw.githubusercontent.com/trishakarma/stocks-sentiment-analysis/refs/heads/main/tweets_labelled_09042020_16072020.csv"
file_path = "tweets_labelled_09042020_16072020.csv"

if not os.path.exists(file_path):
    wget.download(csv_file_path, file_path)
    print("File Downloaded")
else:
    print("File already exists in directory")

File Downloaded


In [9]:
import pandas as pd
df = pd.read_csv(file_path,delimiter=";", usecols=["text", "sentiment"])
df.head()

,text,sentiment
0,"RT @RobertBeadles: Yo💥\nEnter to WIN 1,000 Mon...",positive
1,#SriLanka surcharge on fuel removed!\n⛽📉\nThe ...,negative
2,Net issuance increases to fund fiscal programs...,positive
3,RT @bentboolean: How much of Amazon's traffic ...,positive
4,$AMD Ryzen 4000 desktop CPUs looking ‘great’ a...,positive


In [15]:
%%sql
DROP DATABASE IF EXISTS sentiments;
CREATE DATABASE IF NOT EXISTS sentiments;

1 rows affected.

1 rows affected.

++
||
++
++

In [20]:
%%sql
USE sentiments;
DROP TABLE IF EXISTS tweets;
CREATE TABLE IF NOT EXISTS tweets (
    text TEXT,
    sentiment TEXT,
    embedding VECTOR(384) NOT NULL
);
    

++
||
++
++

In [13]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')

texts = df['text'].tolist()
embeddings = model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)
combined_data = [tuple(row) + (embedding,) for row, embedding in zip(df.values.tolist(), embeddings)]
s2_df = pd.DataFrame(combined_data)

s2_df.rename(columns={0: 'text', 1: 'sentiment', 2: 'embedding'}, inplace=True)

s2_df['embedding'] = s2_df['embedding'].apply(lambda row: np.array(row).astype(np.float32))


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

In [22]:
%%sql
USE sentiments;
TRUNCATE TABLE tweets;

++
||
++
++

In [53]:
import sqlalchemy
from sqlalchemy import create_engine

db_connection = create_engine(connection_url)

s2_df.to_sql(
    "tweets", 
    con=db_connection, 
    schema="sentiments",
    if_exists="append", 
    index=False, 
    chunksize=1000
)

5000

In [54]:
%%sql
USE sentiments;
SELECT * FROM tweets LIMIT 5;

5 rows affected.

+---------------------------------------------------------------------------------------+-----------+------------------------------------------------------------------------+
|                                          text                                         | sentiment |                               embedding                                |
+---------------------------------------------------------------------------------------+-----------+------------------------------------------------------------------------+
|  REPORT: Facebook fails to win over ad boycott organizers $FB https://t.co/csoywrk0MP |  negative | [-0.005619  0.079592 -0.005696  0.00839   0.062008 -0.010623  0.012643 |
|                                                                                       |           |  -0.025267 -0.049222 -0.013745 -0.000705 -0.00853   0.019598  0.003412 |
|                                                                                       |           |   0.009401 -0.009606  0.039632  0.005417 -0.073105  0.004851 -0.076983 |
|                                                                                       |           |  -0.020894 -0.025303  0.066878 -0.036736 -0.059942 -0.093837 -0.001371 |
|                                                                                       |           |   0.00092  -0.000395  0.050307 -0.018315  0.034305  0.047146  0.063126 |
|                                                                                       |           |  -0.034583 -0.028326 -0.096211  0.015231 -0.054399  0.0835    0.036574 |
|                                                                                       |           |  -0.016672  0.018625 -0.009767  0.042895  0.00755   0.042237 -0.018261 |
|                                                                                       |           |  -0.006939  0.010199 -0.003584 -0.002173 -0.024312 -0.05829   0.015875 |
|                                                                                       |           |   0.024391 -0.000877 -0.008737 -0.00256   0.047743 -0.019792 -0.037819 |
|                                                                                       |           |   0.063295 -0.017675  0.069406 -0.114878 -0.033342 -0.047837  0.044782 |
|                                                                                       |           |   0.101619  0.02411  -0.027634  0.066696  0.07668   0.014653 -0.024321 |
|                                                                                       |           |   0.036757  0.037973 -0.002865 -0.049485 -0.118829 -0.026183 -0.025348 |
|                                                                                       |           |   0.08919  -0.035842 -0.065653 -0.047838  0.06793   0.007699 -0.075099 |
|                                                                                       |           |   0.147596  0.029228  0.057915 -0.087362 -0.009974 -0.032468  0.021309 |
|                                                                                       |           |   0.021667  0.09191  -0.045295  0.099803  0.047365 -0.036621  0.039392 |
|                                                                                       |           |  -0.054077  0.023729  0.034142  0.024339  0.044398 -0.010226  0.014254 |
|                                                                                       |           |   0.116256 -0.04382  -0.007524 -0.017847 -0.040083  0.025044  0.104774 |
|                                                                                       |           |   -0.030244 -0.002214  0.067111  0.017944  0.009729 -0.018319 -0.0572  |
|                                                                                       |           |  -0.038732 -0.       -0.014374  0.002517  0.019923 -0.00353   0.058707 |
|                                                                                       |           |   0.057902 -0.051902 -0.084432  0.006844  0.0295